# 『本次練習內容』
運用這幾天所學觀念搭建一個CNN分類器
# 『本次練習目的』
熟悉CNN分類器搭建步驟與原理  
學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) # (50000, 32, 32, 3)

# Normalize Data 寫出標準化的式子
def normalize(X_train, X_test):
    mean = np.mean(X_train, axis=(0,1,2,3))
    std = np.std(X_train, axis=(0,1,2,3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test  = (X_test-mean)/(std+1e-7)
    return X_train, X_test, mean, std

# Normalize Training and Test set
x_train, x_test, mean_train, std_train = normalize(x_train, x_test)

(50000, 32, 32, 3)


In [3]:
# OneHot Label 由 (None, 1) - (None, 10)
# ex label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test  = one_hot.transform(y_test).toarray()

In [5]:
classifier = Sequential()

# 卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), input_shape=(32,32,3), activation='relu', padding='same'))
# 32,3,3 , input_shape=(32,32,3) , activation='relu'
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

# 卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same'))
classifier.add(BatchNormalization())

# Flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100, activation='relu'

# 輸出
classifier.add(Dense(units=10, activation='softmax'))  #因為要輸出10個分類，所以units=10 且用 softmax

# 超過2個分類就要用categorical_crossentropy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)



Epoch 1/100
50000/50000 [==============================] - 5s 96us/step - loss: 1.3436 - accuracy: 0.5375
Epoch 2/100
50000/50000 [==============================] - 3s 69us/step - loss: 0.9255 - accuracy: 0.6752
Epoch 3/100
50000/50000 [==============================] - 4s 73us/step - loss: 0.7436 - accuracy: 0.7401
Epoch 4/100
50000/50000 [==============================] - 3s 68us/step - loss: 0.5996 - accuracy: 0.7897
Epoch 5/100
50000/50000 [==============================] - 4s 71us/step - loss: 0.4688 - accuracy: 0.8362
Epoch 6/100
50000/50000 [==============================] - 3s 70us/step - loss: 0.3644 - accuracy: 0.8731
Epoch 7/100
50000/50000 [==============================] - 3s 67us/step - loss: 0.2805 - accuracy: 0.9022
Epoch 8/100
50000/50000 [==============================] - 3s 69us/step - loss: 0.2102 - accuracy: 0.9271
Epoch 9/100
50000/50000 [==============================] - 3s 68us/step - loss: 0.1801 - accuracy: 0.9380
Epoch 10/100
50000/50000 [==================

50000/50000 [==============================] - 3s 67us/step - loss: 0.0198 - accuracy: 0.9937
Epoch 74/100
50000/50000 [==============================] - 3s 66us/step - loss: 0.0208 - accuracy: 0.9936
Epoch 75/100
50000/50000 [==============================] - 3s 69us/step - loss: 0.0181 - accuracy: 0.9943
Epoch 76/100
50000/50000 [==============================] - 3s 69us/step - loss: 0.0179 - accuracy: 0.9945
Epoch 77/100
50000/50000 [==============================] - 3s 68us/step - loss: 0.0237 - accuracy: 0.9926
Epoch 78/100
50000/50000 [==============================] - 3s 68us/step - loss: 0.0263 - accuracy: 0.9920
Epoch 79/100
50000/50000 [==============================] - 3s 67us/step - loss: 0.0142 - accuracy: 0.9956
Epoch 80/100
50000/50000 [==============================] - 3s 68us/step - loss: 0.0121 - accuracy: 0.9961
Epoch 81/100
50000/50000 [==============================] - ETA: 0s - loss: 0.0185 - accuracy: 0.99 - 3s 66us/step - loss: 0.0186 - accuracy: 0.9940
Epoch 82

# 預測新圖片時，使用的影像前處理要與訓練時相同
((X-mean)/(std+1e-7)): 這裡的mean和std是訓練集的
# 維度如下方示範

In [7]:
input_example = (np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7)
classifier.predict(input_example)

array([[3.4243084e-04, 4.6952007e-08, 3.1102911e-01, 4.2117769e-01,
        2.6483324e-01, 1.9691619e-08, 2.5442046e-07, 2.6159298e-03,
        1.1798797e-06, 1.4729100e-10]], dtype=float32)